In [1]:
import nltk
nltk.download("gutenberg")
from nltk.corpus import gutenberg
import pandas as pd
import numpy as np
data=gutenberg.raw('shakespeare-hamlet.txt')
with open("hamlet.txt","w") as fl:
    fl.write(data)

[nltk_data] Downloading package gutenberg to
[nltk_data]     /home/workstation-p/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [3]:
with open("hamlet.txt","r") as fl:
    text=fl.read().lower()

tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1
total_words

4818

In [4]:
# create input sequences
input_sequences=[]
for line in text.split("\n"):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [6]:
max_sequence_len=max([len(x)for x in input_sequences])

In [7]:
max_sequence_len

14

In [8]:
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))

In [9]:
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]],
      shape=(25732, 14), dtype=int32)

In [10]:
# create predictors and label
import tensorflow as tf
x,y=input_sequences[:,:-1],input_sequences[:,-1]

In [11]:
x,y

(array([[   0,    0,    0, ...,    0,    0,    1],
        [   0,    0,    0, ...,    0,    1,  687],
        [   0,    0,    0, ...,    1,  687,    4],
        ...,
        [   0,    0,    0, ...,  687,    4,   45],
        [   0,    0,    0, ...,    4,   45, 1047],
        [   0,    0,    0, ...,   45, 1047,    4]],
       shape=(25732, 13), dtype=int32),
 array([ 687,    4,   45, ..., 1047,    4,  193],
       shape=(25732,), dtype=int32))

In [12]:
y=tf.keras.utils.to_categorical(y,num_classes=total_words)

In [13]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)

In [ ]:
# Train LSTM RNN
